In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13].values


In [3]:
#Create dummy variables
geography=pd.get_dummies(X["Geography"],drop_first=True)
gender=pd.get_dummies(X['Gender'],drop_first=True)


In [4]:
## Concatenate the Data Frames
X=pd.concat([X,geography,gender],axis=1)


In [5]:
## Drop Unnecessary columns
X=X.drop(['Geography','Gender'],axis=1)

In [41]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [53]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Perform Hyperparameter Optimization with Scikit_learn Wrapper

In [9]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from tensorflow.keras.activations import relu, sigmoid

In [10]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
    model.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid')) 
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [16]:
model = KerasClassifier(build_fn=create_model, verbose=0)

In [19]:
%%time
layers = ([20],(21,40), (45, 30, 15))
activations = ('sigmoid', 'relu')
param_grid = dict(layers=layers, activation=activations, batch_size = (128, 256), epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5)

grid_result = grid.fit(X_train, y_train)

[grid_result.best_score_,grid_result.best_params_]


CPU times: user 41min 18s, sys: 25min 16s, total: 1h 6min 34s
Wall time: 29min 8s


[0.8554999947547912,
 {'activation': 'relu',
  'batch_size': 128,
  'epochs': 30,
  'layers': (45, 30, 15)}]

In [2]:
#!pip install -q -U keras-tuner
import kerastuner as kt

## Perform Hyperparameter Optimization with Kerastuner

In [43]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch


def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                           activation='relu',input_dim=X_train.shape[1]))
    model.add(layers.Dense(1, activation='sigmoid',kernel_initializer= 'glorot_uniform'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [44]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='test_pro')

INFO:tensorflow:Reloading Oracle from existing project my_dir/test_pro/oracle.json


In [45]:
tuner.search_space_summary()

In [49]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - ETA: 5:05 - loss: 0.6780 - accuracy: 0.75 - ETA: 30s - loss: 0.4778 - accuracy: 0.8250 - ETA: 22s - loss: 0.4665 - accuracy: 0.821 - ETA: 15s - loss: 0.4581 - accuracy: 0.815 - ETA: 14s - loss: 0.4585 - accuracy: 0.815 - ETA: 13s - loss: 0.4533 - accuracy: 0.818 - ETA: 13s - loss: 0.4523 - accuracy: 0.817 - ETA: 12s - loss: 0.4457 - accuracy: 0.821 - ETA: 11s - loss: 0.4419 - accuracy: 0.820 - ETA: 11s - loss: 0.4463 - accuracy: 0.820 - ETA: 11s - loss: 0.4439 - accuracy: 0.819 - ETA: 10s - loss: 0.4461 - accuracy: 0.820 - ETA: 10s - loss: 0.4470 - accuracy: 0.818 - ETA: 10s - loss: 0.4449 - accuracy: 0.818 - ETA: 9s - loss: 0.4441 - accuracy: 0.817 - ETA: 9s - loss: 0.4387 - accuracy: 0.81 - ETA: 9s - loss: 0.4357 - accuracy: 0.82 - ETA: 8s - loss: 0.4333 - accuracy: 0.82 - ETA: 8s - loss: 0.4352 - accuracy: 0.82 - ETA: 8s - loss: 0.4341 - accuracy: 0.82 - ETA: 8s - loss: 0.4311 - ac

8000/8000 [==============================] - ETA: 1s - loss: 0.4386 - accuracy: 0.78 - ETA: 3s - loss: 0.2878 - accuracy: 0.90 - ETA: 4s - loss: 0.3182 - accuracy: 0.88 - ETA: 2s - loss: 0.3514 - accuracy: 0.86 - ETA: 1s - loss: 0.3601 - accuracy: 0.84 - ETA: 1s - loss: 0.3403 - accuracy: 0.86 - ETA: 1s - loss: 0.3418 - accuracy: 0.86 - ETA: 0s - loss: 0.3477 - accuracy: 0.85 - ETA: 0s - loss: 0.3474 - accuracy: 0.85 - ETA: 0s - loss: 0.3416 - accuracy: 0.86 - ETA: 0s - loss: 0.3450 - accuracy: 0.85 - ETA: 0s - loss: 0.3482 - accuracy: 0.86 - ETA: 0s - loss: 0.3552 - accuracy: 0.85 - ETA: 0s - loss: 0.3562 - accuracy: 0.85 - ETA: 0s - loss: 0.3572 - accuracy: 0.85 - ETA: 0s - loss: 0.3567 - accuracy: 0.85 - ETA: 0s - loss: 0.3604 - accuracy: 0.85 - ETA: 0s - loss: 0.3593 - accuracy: 0.85 - ETA: 0s - loss: 0.3619 - accuracy: 0.85 - ETA: 0s - loss: 0.3632 - accuracy: 0.85 - ETA: 0s - loss: 0.3609 - accuracy: 0.85 - ETA: 0s - loss: 0.3609 - accuracy: 0.85 - ETA: 0s - loss: 0.3621 - accura

8000/8000 [==============================] - ETA: 1s - loss: 0.1947 - accuracy: 0.93 - ETA: 3s - loss: 0.3776 - accuracy: 0.85 - ETA: 2s - loss: 0.3330 - accuracy: 0.87 - ETA: 2s - loss: 0.3466 - accuracy: 0.86 - ETA: 2s - loss: 0.3302 - accuracy: 0.87 - ETA: 1s - loss: 0.3480 - accuracy: 0.86 - ETA: 1s - loss: 0.3564 - accuracy: 0.85 - ETA: 1s - loss: 0.3601 - accuracy: 0.85 - ETA: 1s - loss: 0.3528 - accuracy: 0.86 - ETA: 1s - loss: 0.3505 - accuracy: 0.86 - ETA: 1s - loss: 0.3450 - accuracy: 0.86 - ETA: 1s - loss: 0.3464 - accuracy: 0.86 - ETA: 1s - loss: 0.3492 - accuracy: 0.86 - ETA: 1s - loss: 0.3522 - accuracy: 0.86 - ETA: 1s - loss: 0.3530 - accuracy: 0.86 - ETA: 0s - loss: 0.3507 - accuracy: 0.86 - ETA: 0s - loss: 0.3528 - accuracy: 0.85 - ETA: 0s - loss: 0.3535 - accuracy: 0.85 - ETA: 0s - loss: 0.3554 - accuracy: 0.85 - ETA: 0s - loss: 0.3528 - accuracy: 0.85 - ETA: 0s - loss: 0.3492 - accuracy: 0.85 - ETA: 0s - loss: 0.3475 - accuracy: 0.85 - ETA: 0s - loss: 0.3438 - accura

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - ETA: 3:11 - loss: 0.6924 - accuracy: 0.62 - ETA: 12s - loss: 0.7222 - accuracy: 0.4646 - ETA: 7s - loss: 0.7050 - accuracy: 0.536 - ETA: 5s - loss: 0.6945 - accuracy: 0.55 - ETA: 3s - loss: 0.6859 - accuracy: 0.58 - ETA: 3s - loss: 0.6763 - accuracy: 0.61 - ETA: 2s - loss: 0.6639 - accuracy: 0.64 - ETA: 1s - loss: 0.6569 - accuracy: 0.65 - ETA: 1s - loss: 0.6477 - accuracy: 0.67 - ETA: 1s - loss: 0.6375 - accuracy: 0.69 - ETA: 1s - loss: 0.6324 - accuracy: 0.69 - ETA: 0s - loss: 0.6263 - accuracy: 0.70 - ETA: 0s - loss: 0.6183 - accuracy: 0.71 - ETA: 0s - loss: 0.6119 - accuracy: 0.72 - ETA: 0s - loss: 0.6067 - accuracy: 0.72 - ETA: 0s - loss: 0.6010 - accuracy: 0.73 - ETA: 0s - loss: 0.5973 - accuracy: 0.73 - ETA: 0s - loss: 0.5928 - accuracy: 0.74 - 2s 259us/sample - loss: 0.5925 - accuracy: 0.7416 - val_loss: 0.5154 - val_accuracy: 0.8065
Epoch 2/5
8000/8000 [======================

8000/8000 [==============================] - ETA: 1s - loss: 0.4024 - accuracy: 0.84 - ETA: 1s - loss: 0.4519 - accuracy: 0.80 - ETA: 1s - loss: 0.4588 - accuracy: 0.80 - ETA: 1s - loss: 0.4439 - accuracy: 0.81 - ETA: 1s - loss: 0.4485 - accuracy: 0.80 - ETA: 1s - loss: 0.4477 - accuracy: 0.81 - ETA: 1s - loss: 0.4506 - accuracy: 0.80 - ETA: 1s - loss: 0.4502 - accuracy: 0.80 - ETA: 1s - loss: 0.4508 - accuracy: 0.80 - ETA: 1s - loss: 0.4587 - accuracy: 0.79 - ETA: 1s - loss: 0.4559 - accuracy: 0.79 - ETA: 1s - loss: 0.4526 - accuracy: 0.80 - ETA: 0s - loss: 0.4485 - accuracy: 0.80 - ETA: 0s - loss: 0.4438 - accuracy: 0.80 - ETA: 0s - loss: 0.4427 - accuracy: 0.80 - ETA: 0s - loss: 0.4417 - accuracy: 0.80 - ETA: 0s - loss: 0.4401 - accuracy: 0.80 - ETA: 0s - loss: 0.4420 - accuracy: 0.80 - ETA: 0s - loss: 0.4392 - accuracy: 0.80 - ETA: 0s - loss: 0.4388 - accuracy: 0.80 - ETA: 0s - loss: 0.4380 - accuracy: 0.80 - ETA: 0s - loss: 0.4384 - accuracy: 0.80 - ETA: 0s - loss: 0.4358 - accura

...

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - ETA: 2:48 - loss: 0.7408 - accuracy: 0.46 - ETA: 11s - loss: 0.6821 - accuracy: 0.5917 - ETA: 7s - loss: 0.6479 - accuracy: 0.666 - ETA: 4s - loss: 0.6188 - accuracy: 0.70 - ETA: 4s - loss: 0.6036 - accuracy: 0.71 - ETA: 3s - loss: 0.5826 - accuracy: 0.73 - ETA: 3s - loss: 0.5743 - accuracy: 0.73 - ETA: 2s - loss: 0.5563 - accuracy: 0.74 - ETA: 2s - loss: 0.5480 - accuracy: 0.75 - ETA: 1s - loss: 0.5297 - accuracy: 0.76 - ETA: 1s - loss: 0.5237 - accuracy: 0.76 - ETA: 1s - loss: 0.5138 - accuracy: 0.77 - ETA: 0s - loss: 0.5056 - accuracy: 0.77 - ETA: 0s - loss: 0.5025 - accuracy: 0.77 - ETA: 0s - loss: 0.4980 - accuracy: 0.77 - ETA: 0s - loss: 0.4943 - accuracy: 0.78 - ETA: 0s - loss: 0.4898 - accuracy: 0.78 - ETA: 0s - loss: 0.4856 - accuracy: 0.78 - ETA: 0s - loss: 0.4809 - accuracy: 0.78 - ETA: 0s - loss: 0.4755 - accuracy: 0.79 - ETA: 0s - loss: 0.4724 - accuracy: 0.79 - ETA: 0s -

Epoch 2/5
8000/8000 [==============================] - ETA: 3s - loss: 0.3621 - accuracy: 0.90 - ETA: 1s - loss: 0.4163 - accuracy: 0.81 - ETA: 0s - loss: 0.4256 - accuracy: 0.80 - ETA: 0s - loss: 0.4195 - accuracy: 0.81 - ETA: 0s - loss: 0.4214 - accuracy: 0.81 - ETA: 0s - loss: 0.4091 - accuracy: 0.82 - ETA: 0s - loss: 0.4112 - accuracy: 0.82 - ETA: 0s - loss: 0.4083 - accuracy: 0.82 - ETA: 0s - loss: 0.4070 - accuracy: 0.82 - ETA: 0s - loss: 0.4029 - accuracy: 0.83 - ETA: 0s - loss: 0.4038 - accuracy: 0.83 - ETA: 0s - loss: 0.4048 - accuracy: 0.82 - ETA: 0s - loss: 0.4015 - accuracy: 0.83 - ETA: 0s - loss: 0.4002 - accuracy: 0.83 - ETA: 0s - loss: 0.3991 - accuracy: 0.83 - ETA: 0s - loss: 0.3991 - accuracy: 0.83 - ETA: 0s - loss: 0.3997 - accuracy: 0.83 - ETA: 0s - loss: 0.3996 - accuracy: 0.83 - 1s 135us/sample - loss: 0.3982 - accuracy: 0.8319 - val_loss: 0.3794 - val_accuracy: 0.8500
Epoch 3/5
8000/8000 [==============================] - ETA: 2s - loss: 0.4249 - accuracy: 0.87 - 

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - ETA: 2:56 - loss: 0.6887 - accuracy: 0.56 - ETA: 12s - loss: 0.5970 - accuracy: 0.7679 - ETA: 7s - loss: 0.5594 - accuracy: 0.780 - ETA: 4s - loss: 0.5375 - accuracy: 0.77 - ETA: 2s - loss: 0.5103 - accuracy: 0.78 - ETA: 2s - loss: 0.4985 - accuracy: 0.79 - ETA: 1s - loss: 0.4908 - accuracy: 0.79 - ETA: 1s - loss: 0.4856 - accuracy: 0.79 - ETA: 1s - loss: 0.4792 - accuracy: 0.79 - ETA: 1s - loss: 0.4741 - accuracy: 0.79 - ETA: 0s - loss: 0.4722 - accuracy: 0.79 - ETA: 0s - loss: 0.4695 - accuracy: 0.79 - ETA: 0s - loss: 0.4647 - accuracy: 0.80 - ETA: 0s - loss: 0.4626 - accuracy: 0.80 - ETA: 0s - loss: 0.4602 - accuracy: 0.80 - ETA: 0s - loss: 0.4538 - accuracy: 0.80 - ETA: 0s - loss: 0.4505 - accuracy: 0.80 - ETA: 0s - loss: 0.4475 - accuracy: 0.80 - 2s 255us/sample - loss: 0.4441 - accuracy: 0.8089 - val_loss: 0.3929 - val_accuracy: 0.8450
Epoch 2/5
8000/8000 [======================

8000/8000 [==============================] - ETA: 2s - loss: 0.4163 - accuracy: 0.87 - ETA: 1s - loss: 0.3351 - accuracy: 0.87 - ETA: 1s - loss: 0.3273 - accuracy: 0.87 - ETA: 1s - loss: 0.3250 - accuracy: 0.87 - ETA: 0s - loss: 0.3348 - accuracy: 0.87 - ETA: 0s - loss: 0.3467 - accuracy: 0.86 - ETA: 0s - loss: 0.3456 - accuracy: 0.86 - ETA: 0s - loss: 0.3462 - accuracy: 0.86 - ETA: 0s - loss: 0.3459 - accuracy: 0.86 - ETA: 0s - loss: 0.3467 - accuracy: 0.86 - ETA: 0s - loss: 0.3538 - accuracy: 0.85 - ETA: 0s - loss: 0.3572 - accuracy: 0.85 - ETA: 0s - loss: 0.3551 - accuracy: 0.85 - ETA: 0s - loss: 0.3594 - accuracy: 0.85 - ETA: 0s - loss: 0.3611 - accuracy: 0.85 - ETA: 0s - loss: 0.3630 - accuracy: 0.85 - ETA: 0s - loss: 0.3628 - accuracy: 0.85 - ETA: 0s - loss: 0.3601 - accuracy: 0.85 - ETA: 0s - loss: 0.3602 - accuracy: 0.85 - ETA: 0s - loss: 0.3581 - accuracy: 0.85 - ETA: 0s - loss: 0.3546 - accuracy: 0.85 - ETA: 0s - loss: 0.3524 - accuracy: 0.85 - ETA: 0s - loss: 0.3541 - accura

INFO:tensorflow:Oracle triggered exit


In [50]:
models = tuner.get_best_models(num_models=2)

In [51]:
tuner.results_summary()